In [ ]:
import os
import csv
from collections import defaultdict
from PIL import Image

def calculate_iou(box1, box2):
    # No changes needed here
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    x_left = max(x1 - w1 / 2, x2 - w2 / 2)
    y_top = max(y1 - h1 / 2, y2 - h2 / 2)
    x_right = min(x1 + w1 / 2, x2 + w2 / 2)
    y_bottom = min(y1 + h1 / 2, y2 + h2 / 2)
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = w1 * h1
    box2_area = w2 * h2
    iou = intersection_area / (box1_area + box2_area - intersection_area)
    return iou

def process_files(images_folder, gt_folder, prediction_folder):
    annotations = defaultdict(list)
    matched_predictions = set()
    for filename in os.listdir(images_folder):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            img = Image.open(os.path.join(images_folder, filename))
            img_width, img_height = img.size
            gt_file = os.path.join(gt_folder, filename.rsplit('.', 1)[0] + ".txt")
            pred_file = os.path.join(prediction_folder, filename.rsplit('.', 1)[0] + ".txt")
            with open(gt_file, "r") as f:
                gt_annotations = [list(map(float, line.strip().split())) for line in f]
            with open(pred_file, "r") as f:
                pred_annotations = [list(map(float, line.strip().split())) for line in f]
            for gt_obj in gt_annotations:
                gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height = gt_obj
                matched_pred = None
                max_iou = 0
                for pred_obj in pred_annotations:
                    iou = calculate_iou((gt_x_center, gt_y_center, gt_obj_width, gt_obj_height), pred_obj[1:5])
                    if iou > max_iou:
                        max_iou = iou
                        matched_pred = pred_obj
                if matched_pred and max_iou >= 0.5:
                    annotations[filename].append([filename, img_width, img_height, gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height] + matched_pred)
                    matched_predictions.add(tuple(matched_pred))
                else:
                    annotations[filename].append([filename, img_width, img_height, gt_obj_id, gt_x_center, gt_y_center, gt_obj_width, gt_obj_height, None, None, None, None, None])  # Use None for unmatched predictions
            for pred_obj in pred_annotations:
                if tuple(pred_obj) not in matched_predictions:
                    annotations[filename].append([filename, img_width, img_height, None, None, None, None, None] + pred_obj)  # Correctly handle unmatched predictions
    return annotations

def write_to_csv(annotations, output_csv):
    with open(output_csv, "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        headers = ["filename", "image_width", "image_height", "gt_obj_id", "gt_x_center", "gt_y_center", "gt_obj_width", "gt_obj_height", "pred_obj_id", "pred_x_center", "pred_y_center", "pred_obj_width", "pred_obj_height", "conf_score","objectness_score"]
        # Dynamically adjust class confidence score headers based on the data
        max_classes = max(len(ann) - 15 for anns in annotations.values() for ann in anns)  # Adjust index based on actual data structure
        class_headers = [f"class_{i}_conf_score" for i in range(max_classes)]
        csv_writer.writerow(headers + class_headers)
        for filename, ann_list in annotations.items():
            for ann in ann_list:
                csv_writer.writerow(ann + [None] * (max_classes - (len(ann) - 14)))  # Fill missing class confidence scores with None

# Define paths
images_folder = "test/images/"
gt_folder = "test/labels/"
prediction_folder = "test/pred_test/"
output_csv = "test/test_challenge.csv"

# Process files and write to CSV
annotations = process_files(images_folder, gt_folder, prediction_folder)
write_to_csv(annotations, output_csv)